In [1]:
# 패키지 임포트
import selenium
from selenium import webdriver
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import time
from tqdm import tqdm_notebook
import urllib
from urllib.request import urlopen

In [33]:
df = pd.read_csv('./서울시동작구(업소URL추가).csv',encoding='cp949',index_col=0)
df

,상세영업상태명,전화번호,소재지면적,소재지우편번호,지번주소,도로명주소,도로명우편번호,사업장명,최종수정일자,데이터갱신구분,데이터갱신일자,업태구분명,좌표정보(X),좌표정보(Y),위생업태명,검색명,업소URL
0,영업,NaN,69.34,156800,서울특별시 동작구 노량진동 57-1 메가스터디타워,서울특별시 동작구 노량진로 140 메가스터디타워 상가동층 115호 (노량진동),6922.0,골든볼9 몬스터(지훈식당),2.020000e+13,U,2021-04-29 2:40,한식,194772.9754,445623.0802,한식,동작구 노량진동골든볼9 몬스터(지훈식당),-
1,영업,NaN,26.00,156871,서울특별시 동작구 상도1동 804-1,서울특별시 동작구 상도로38길 3-20 1층 102호 (상도1동),7032.0,네모푸드 상도점,2.020000e+13,I,2021-02-05 0:23,한식,195365.9867,444373.1854,한식,동작구 상도1동네모푸드 상도점,-
2,영업,NaN,64.74,156849,서울특별시 동작구 신대방동 395-70번지 전문건설회관 지하1층,서울특별시 동작구 보라매로5길 15 (신대방동전문건설회관 지하1층),NaN,늘,2.010000e+13,I,2018-08-31 23:59,한식,193232.0279,443314.0572,한식,동작구 신대방동늘,1380778265
3,영업,NaN,30.00,156843,서울특별시 동작구 상도동 355-8,서울특별시 동작구 상도로 122 1층 (상도동),6959.0,필드키친,2.020000e+13,I,2021-12-08 0:22,한식,194077.5201,444347.4355,한식,동작구 상도동필드키친,-
4,영업,02 8156555,16.01,156804,서울특별시 동작구 노량진동 261-10,서울특별시 동작구 노량진로8길 28 (노량진동),6930.0,백가네해물칼국수,2.020000e+13,U,2021-12-14 2:40,한식,194347.0685,445597.5115,한식,동작구 노량진동백가네해물칼국수,18752808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,영업,02 5889255,19.42,156879,서울특별시 동작구 사당동 206-12번지 (지상1층),서울특별시 동작구 사당로4길 13 (사당동(지상1층)),7030.0,강남식당,2.020000e+13,I,2018-08-31 23:59,김밥(도시락),196942.7009,442766.0717,김밥(도시락),동작구 사당동강남식당,994992031
332,영업,8133838,23.20,156843,서울특별시 동작구 상도동 250-16,서울특별시 동작구 성대로5길 2 (상도동),6960.0,김밥이맛있는집,2.020000e+13,U,2021-12-09 2:40,김밥(도시락),194027.1309,444137.2806,김밥(도시락),동작구 상도동김밥이맛있는집,18696397
333,영업,NaN,57.81,156030,서울특별시 동작구 상도동 507-3번지,서울특별시 동작구 상도로61길 72 지하1층 B102호 (상도동),7027.0,추억과김밥,2.020000e+13,I,2018-08-31 23:59,김밥(도시락),196284.0741,443586.5440,김밥(도시락),동작구 상도동추억과김밥,18698641
334,영업,28240216,29.50,156030,서울특별시 동작구 상도동 535,서울특별시 동작구 상도로30길 40 1층 101호 (상도동 두산위브트레지움아파트상가),6964.0,해피트리,2.020000e+13,U,2020-10-24 2:40,김밥(도시락),194870.3844,444631.1127,김밥(도시락),동작구 상도동해피트리,1842791728


In [17]:
kind = pd.read_csv('./서울시 착한가격음식점(종합).csv',encoding='cp949',index_col=0)
kind = kind[kind['행정구'] == '동작구']
kind.식당아이디
kind.rename(columns={'업소명':'사업장명'},inplace=True)

In [23]:
# 착한가격업소 가게와 중복되는 일반음식점 제외
kind.merge(df,how='inner',on='사업장명')

,업소아이디,사업장명,분류코드,분류코드명,업소 주소,업소 전화번호,찾아오시는 길,업소정보,자랑거리,추천수,...,도로명우편번호,최종수정일자,데이터갱신구분,데이터갱신일자,업태구분명,좌표정보(X),좌표정보(Y),위생업태명,검색명_y,업소URL
0,9285,이고시다,1,한식,서울특별시 동작구 여의대방로22길 98 (신대방동),821-7007,NaN,NaN,저렴한 가격 깨끗한 위생,0,...,NaN,2.020000e+13,U,2020-11-13 2:40,한식,NaN,NaN,한식,동작구 신대방동이고시다,12859870
1,9323,남원추어탕,1,한식,서울특별시 동작구 동작대로15길 15 (사당동),522-6243,NaN,저렴한 가격 깨끗한 위생,저렴한 가격 깨끗한 위생,0,...,6943.0,2.020000e+13,U,2020-11-13 2:40,한식,193273.4840,445011.2501,한식,동작구 대방동남원추어탕,37943784
2,9590,남원추어탕,1,한식,서울특별시 서울 동작구 여의대방로 250 지하1층 14호,02-812-2047,NaN,저렴한 가격 깨끗한 위생,저렴한 가격 깨끗한 위생,0,...,6943.0,2.020000e+13,U,2020-11-13 2:40,한식,193273.4840,445011.2501,한식,동작구 대방동남원추어탕,37943784
3,9286,삼호정,1,한식,서울특별시 동작구 여의대방로24길 50 (신대방동),825-6023,NaN,null,저렴한 가격 깨끗한 위생,0,...,7056.0,2.020000e+13,U,2020-11-04 2:40,한식,193121.1922,443973.8993,한식,동작구 신대방동삼호정,1493009891
4,9541,달길포차,1,한식,서울특별시 동작구 사당로30길 63 1층,없음,NaN,저렴한 가격 깨끗한 위생,저렴한 가격 깨끗한 위생,0,...,7014.0,2.020000e+13,U,2019-09-22 2:40,한식,198247.6850,442196.3500,한식,동작구 사당동달길포차,1442408791
5,9546,준호가만드는부대찌개,1,한식,서울특별시 동작구 흑석로13마길 61 1층,02-6083-8247,NaN,저렴한 가격 깨끗한 위생,저렴한 가격 깨끗한 위생,0,...,6910.0,2.020000e+13,U,2019-06-01 2:40,한식,196296.6170,445023.5210,한식,동작구 흑석동준호가만드는부대찌개,-
6,9549,호순네집,1,한식,서울특별시 동작구 등용로 107 1층,02-812-9185,NaN,저렴한 가격 깨끗한 위생,저렴한 가격 깨끗한 위생,0,...,6942.0,2.020000e+13,I,2019-02-14 2:21,한식,193907.5527,445386.4642,한식,동작구 대방동호순네집,1405815470
7,9523,제주왕뚜껑생고기,1,한식,서울특별시 서울 동작구 등용로 110,02-826-1342,NaN,저렴한 가격 깨끗한 위생,저렴한 가격 깨끗한 위생,0,...,6932.0,2.020000e+13,U,2021-09-09 2:40,한식,193938.7442,445417.2466,한식,동작구 대방동제주왕뚜껑생고기,-
8,9550,태원,2,중식,서울특별시 동작구 등용로 81 1층,02-815-5155,NaN,저렴한 가격 깨끗한 위생,저렴한 가격 깨끗한 위생,0,...,6947.0,2.010000e+13,I,2018-08-31 23:59,중국식,193920.9645,445135.8383,중국식,동작구 대방동태원,18736549


In [31]:
df[df['사업장명']=='태원']

,상세영업상태명,전화번호,소재지면적,소재지우편번호,지번주소,도로명주소,도로명우편번호,사업장명,최종수정일자,데이터갱신구분,데이터갱신일자,업태구분명,좌표정보(X),좌표정보(Y),위생업태명,검색명,업소URL
309,영업,02 8175154,91.18,156807,서울특별시 동작구 대방동 14-9번지,서울특별시 동작구 등용로 81 (대방동),6947.0,태원,2.010000e+13,I,2018-08-31 23:59,중국식,193920.9645,445135.8383,중국식,동작구 대방동태원,18736549


In [34]:
# 국민우동집,대영식당 주소동일 중복 데이터 확인
# 제거 후 일반음식점 메뉴가격 크롤링
df.drop([139,202,213,170,5,219,125,309],inplace=True)
df

,상세영업상태명,전화번호,소재지면적,소재지우편번호,지번주소,도로명주소,도로명우편번호,사업장명,최종수정일자,데이터갱신구분,데이터갱신일자,업태구분명,좌표정보(X),좌표정보(Y),위생업태명,검색명,업소URL
0,영업,NaN,69.34,156800,서울특별시 동작구 노량진동 57-1 메가스터디타워,서울특별시 동작구 노량진로 140 메가스터디타워 상가동층 115호 (노량진동),6922.0,골든볼9 몬스터(지훈식당),2.020000e+13,U,2021-04-29 2:40,한식,194772.9754,445623.0802,한식,동작구 노량진동골든볼9 몬스터(지훈식당),-
1,영업,NaN,26.00,156871,서울특별시 동작구 상도1동 804-1,서울특별시 동작구 상도로38길 3-20 1층 102호 (상도1동),7032.0,네모푸드 상도점,2.020000e+13,I,2021-02-05 0:23,한식,195365.9867,444373.1854,한식,동작구 상도1동네모푸드 상도점,-
2,영업,NaN,64.74,156849,서울특별시 동작구 신대방동 395-70번지 전문건설회관 지하1층,서울특별시 동작구 보라매로5길 15 (신대방동전문건설회관 지하1층),NaN,늘,2.010000e+13,I,2018-08-31 23:59,한식,193232.0279,443314.0572,한식,동작구 신대방동늘,1380778265
3,영업,NaN,30.00,156843,서울특별시 동작구 상도동 355-8,서울특별시 동작구 상도로 122 1층 (상도동),6959.0,필드키친,2.020000e+13,I,2021-12-08 0:22,한식,194077.5201,444347.4355,한식,동작구 상도동필드키친,-
4,영업,02 8156555,16.01,156804,서울특별시 동작구 노량진동 261-10,서울특별시 동작구 노량진로8길 28 (노량진동),6930.0,백가네해물칼국수,2.020000e+13,U,2021-12-14 2:40,한식,194347.0685,445597.5115,한식,동작구 노량진동백가네해물칼국수,18752808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,영업,02 5889255,19.42,156879,서울특별시 동작구 사당동 206-12번지 (지상1층),서울특별시 동작구 사당로4길 13 (사당동(지상1층)),7030.0,강남식당,2.020000e+13,I,2018-08-31 23:59,김밥(도시락),196942.7009,442766.0717,김밥(도시락),동작구 사당동강남식당,994992031
332,영업,8133838,23.20,156843,서울특별시 동작구 상도동 250-16,서울특별시 동작구 성대로5길 2 (상도동),6960.0,김밥이맛있는집,2.020000e+13,U,2021-12-09 2:40,김밥(도시락),194027.1309,444137.2806,김밥(도시락),동작구 상도동김밥이맛있는집,18696397
333,영업,NaN,57.81,156030,서울특별시 동작구 상도동 507-3번지,서울특별시 동작구 상도로61길 72 지하1층 B102호 (상도동),7027.0,추억과김밥,2.020000e+13,I,2018-08-31 23:59,김밥(도시락),196284.0741,443586.5440,김밥(도시락),동작구 상도동추억과김밥,18698641
334,영업,28240216,29.50,156030,서울특별시 동작구 상도동 535,서울특별시 동작구 상도로30길 40 1층 101호 (상도동 두산위브트레지움아파트상가),6964.0,해피트리,2.020000e+13,U,2020-10-24 2:40,김밥(도시락),194870.3844,444631.1127,김밥(도시락),동작구 상도동해피트리,1842791728


In [53]:
df = df[df['업소URL'] != '-']
# df = df.drop(['level_0'],axis=1)
# df = df.drop(['index'],axis=1)
df.reset_index(inplace=True)

In [55]:
df = df.drop(['index'],axis=1)
df

,상세영업상태명,전화번호,소재지면적,소재지우편번호,지번주소,도로명주소,도로명우편번호,사업장명,최종수정일자,데이터갱신구분,데이터갱신일자,업태구분명,좌표정보(X),좌표정보(Y),위생업태명,검색명,업소URL
0,영업,NaN,64.74,156849,서울특별시 동작구 신대방동 395-70번지 전문건설회관 지하1층,서울특별시 동작구 보라매로5길 15 (신대방동전문건설회관 지하1층),NaN,늘,2.010000e+13,I,2018-08-31 23:59,한식,193232.0279,443314.0572,한식,동작구 신대방동늘,1380778265
1,영업,02 8156555,16.01,156804,서울특별시 동작구 노량진동 261-10,서울특별시 동작구 노량진로8길 28 (노량진동),6930.0,백가네해물칼국수,2.020000e+13,U,2021-12-14 2:40,한식,194347.0685,445597.5115,한식,동작구 노량진동백가네해물칼국수,18752808
2,영업,02 5357866,96.40,156816,서울특별시 동작구 사당동 138-3 102호,서울특별시 동작구 동작대로27다길 26 (사당동102호),7008.0,구구향훠궈,2.020000e+13,U,2020-12-11 2:40,한식,198181.9469,442684.3355,한식,동작구 사당동구구향훠궈,31148801
3,영업,NaN,146.20,156030,서울특별시 동작구 상도동 500-16 1층,서울특별시 동작구 상도로 383 (상도동 1층),7027.0,놀부자 정육식당,2.020000e+13,U,2021-10-31 2:40,한식,195961.5667,443647.0864,한식,동작구 상도동놀부자 정육식당,1459730247
4,영업,02 34871001,84.76,156821,서울특별시 동작구 사당동 265-10번지 외1필지 (지하1층),서울특별시 동작구 사당로 220 (사당동외1필지 (지하1층)),7010.0,할미가,2.020000e+13,I,2018-08-31 23:59,한식,197472.4706,442415.4760,한식,동작구 사당동할미가,1155852774
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266,영업,NaN,43.00,156846,서울특별시 동작구 상도동 299-58,서울특별시 동작구 국사봉길 74 1층 106호 (상도동),7052.0,차고집냉면 동작점,2.020000e+13,U,2021-12-06 22:07,냉면집,193790.5229,443823.8910,NaN,동작구 상도동차고집냉면 동작점,1212136013
267,영업,02 5889255,19.42,156879,서울특별시 동작구 사당동 206-12번지 (지상1층),서울특별시 동작구 사당로4길 13 (사당동(지상1층)),7030.0,강남식당,2.020000e+13,I,2018-08-31 23:59,김밥(도시락),196942.7009,442766.0717,김밥(도시락),동작구 사당동강남식당,994992031
268,영업,8133838,23.20,156843,서울특별시 동작구 상도동 250-16,서울특별시 동작구 성대로5길 2 (상도동),6960.0,김밥이맛있는집,2.020000e+13,U,2021-12-09 2:40,김밥(도시락),194027.1309,444137.2806,김밥(도시락),동작구 상도동김밥이맛있는집,18696397
269,영업,NaN,57.81,156030,서울특별시 동작구 상도동 507-3번지,서울특별시 동작구 상도로61길 72 지하1층 B102호 (상도동),7027.0,추억과김밥,2.020000e+13,I,2018-08-31 23:59,김밥(도시락),196284.0741,443586.5440,김밥(도시락),동작구 상도동추억과김밥,18698641


In [56]:
# test
urlnum = df['업소URL'][0]
url = f'https://pcmap.place.naver.com/restaurant/{urlnum}/menu/list'
url
driver = webdriver.Chrome('./driver/chromedriver')
driver.get(url)
# time.sleep(3)
html = urlopen(driver.current_url)
bs_obj = BeautifulSoup(html,'html.parser')

menu = bs_obj.find('span',{'class':'_3yfZ1'}).text
price = bs_obj.find('div',{'class':'_3qFuX'}).text

print(menu,price)

C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_56368/2416202754.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./driver/chromedriver')


된장찌개 7,000원


In [57]:
len(df)

271

In [60]:
menu_list =[]
price_list = []
for n in tqdm_notebook(range(len(df))) :
    try :
        urlnum = df['업소URL'][n]
        url = f'https://pcmap.place.naver.com/restaurant/{urlnum}/menu/list'
        
        driver = webdriver.Chrome('./driver/chromedriver')
        driver.get(url)
        time.sleep(3)
        html = urlopen(driver.current_url)
        bs_obj = BeautifulSoup(html,'html.parser')

        menu = bs_obj.find('span',{'class':'_3yfZ1'}).text
        price = bs_obj.find('div',{'class':'_3qFuX'}).text
        
        menu_list.append(menu)
        price_list.append(price)
        
    except :
        menu_list.append("-")
        price_list.append("-")

C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_56368/2349529281.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for n in tqdm_notebook(range(len(df))) :


  0%|          | 0/271 [00:00<?, ?it/s]

C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_56368/2349529281.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./driver/chromedriver')


In [62]:
df['대표메뉴'] = menu_list
df['가격정보'] = price_list
df.head(10)

,상세영업상태명,전화번호,소재지면적,소재지우편번호,지번주소,도로명주소,도로명우편번호,사업장명,최종수정일자,데이터갱신구분,데이터갱신일자,업태구분명,좌표정보(X),좌표정보(Y),위생업태명,검색명,업소URL,대표메뉴,가격정보
0,영업,NaN,64.74,156849,서울특별시 동작구 신대방동 395-70번지 전문건설회관 지하1층,서울특별시 동작구 보라매로5길 15 (신대방동전문건설회관 지하1층),NaN,늘,2.010000e+13,I,2018-08-31 23:59,한식,193232.0279,443314.0572,한식,동작구 신대방동늘,1380778265,된장찌개,"7,000원"
1,영업,02 8156555,16.01,156804,서울특별시 동작구 노량진동 261-10,서울특별시 동작구 노량진로8길 28 (노량진동),6930.0,백가네해물칼국수,2.020000e+13,U,2021-12-14 2:40,한식,194347.0685,445597.5115,한식,동작구 노량진동백가네해물칼국수,18752808,해물칼국수,"8,000원"
2,영업,02 5357866,96.40,156816,서울특별시 동작구 사당동 138-3 102호,서울특별시 동작구 동작대로27다길 26 (사당동102호),7008.0,구구향훠궈,2.020000e+13,U,2020-12-11 2:40,한식,198181.9469,442684.3355,한식,동작구 사당동구구향훠궈,31148801,무한리필 훠궈,"19,900원"
3,영업,NaN,146.20,156030,서울특별시 동작구 상도동 500-16 1층,서울특별시 동작구 상도로 383 (상도동 1층),7027.0,놀부자 정육식당,2.020000e+13,U,2021-10-31 2:40,한식,195961.5667,443647.0864,한식,동작구 상도동놀부자 정육식당,1459730247,숯불 무한리필,"13,900원"
4,영업,02 34871001,84.76,156821,서울특별시 동작구 사당동 265-10번지 외1필지 (지하1층),서울특별시 동작구 사당로 220 (사당동외1필지 (지하1층)),7010.0,할미가,2.020000e+13,I,2018-08-31 23:59,한식,197472.4706,442415.4760,한식,동작구 사당동할미가,1155852774,삼합소,"35,000원"
5,영업,NaN,11.40,156800,서울특별시 동작구 노량진동 13-6,서울특별시 동작구 노들로 674 지상1층 매점7호 (노량진동),6900.0,우주매점,2.020000e+13,U,2021-01-10 2:40,한식,194346.1567,445823.5009,한식,동작구 노량진동우주매점,1827675196,-,-
6,영업,02 8126842,14.04,156800,서울특별시 동작구 노량진동 85-6번지,서울특별시 동작구 노량진로14가길 14 (노량진동),6923.0,맘스터치,2.010000e+13,I,2018-08-31 23:59,한식,194909.2496,445592.4636,한식,동작구 노량진동맘스터치,33821442,트리플딥치즈싸이버거,변동
7,영업,02 8265248,56.68,156030,서울특별시 동작구 상도동 502-11,서울특별시 동작구 상도로61길 21 (상도동),7027.0,고기스토리,2.020000e+13,U,2021-09-29 2:40,한식,196046.2104,443658.1123,한식,동작구 상도동고기스토리,32848774,철판제육,"7,000원"
8,영업,NaN,28.75,156857,서울특별시 동작구 흑석동 54-142번지 제1호 1층,서울특별시 동작구 서달로14나길 27 (흑석동 제1호 1층),6980.0,흑수돈(흑석동수제돈까스),2.010000e+13,I,2018-08-31 23:59,한식,196673.4987,444970.7665,한식,동작구 흑석동흑수돈(흑석동수제돈까스),33492432,돈까스,"7,500원"
9,영업,02 5960500,103.00,156816,서울특별시 동작구 사당동 139-84번지,서울특별시 동작구 동작대로27가길 5 지하1층 (사당동),7008.0,브로이제트3호점,2.020000e+13,U,2020-05-30 2:40,한식,198274.6926,442795.5984,한식,동작구 사당동브로이제트3호점,1096237126,기네스,"5,900원"


In [20]:
df.drop(['index'],axis=1,inplace=True)

C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_61228/932265166.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['index'],axis=1,inplace=True)


In [64]:
df.to_csv('./서울시 동작구 일반음식점(메뉴가격 추가).csv',encoding='cp949')